In [1]:
import h5py
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
df_train = h5py.File("./data/X_train.h5", "r")
x_train = df_train["features"][:183143]
x_test  = df_train["features"][183143:]

In [3]:
y_train = pd.read_csv("./data/y_train.csv").values[:183143, 1].squeeze()
y_test = pd.read_csv("./data/y_train.csv").values[183143:, 1].squeeze()

In [4]:
def get_waves(x_train):
    freq  = 125*np.fft.fftfreq(x_train.shape[1])
    eeg   = np.abs(np.fft.fft(x_train))
    delta = np.sum(eeg[:, (freq > 0.5) & (freq <= 4)], axis = 1)
    theta = np.sum(eeg[:, (freq > 4) & (freq <= 7)], axis = 1)
    alpha = np.sum(eeg[:, (freq > 7) & (freq <= 13)], axis = 1)
    beta  = np.sum(eeg[:, (freq > 13) & (freq <= 30)], axis = 1)
    gamma = np.sum(eeg[:, (freq > 30)], axis = 1)
    return np.array([delta, theta, alpha, beta, gamma])

In [5]:
features = []
features.append(x_train[:, :11])
features = np.array(features)[0]

In [6]:
waves = get_waves(x_train[:, 11:])

MemoryError: 

In [7]:
waves = np.transpose(waves)

In [8]:
features_train = np.concatenate([features, waves], axis = 1)

In [9]:
features = []
features.append(x_test[:, :11])
features = np.array(features)[0]
waves    = get_waves(x_test[:, 11:])
waves    = np.transpose(waves)
features = np.concatenate([features, waves], axis = 1)
features = preprocessing.scale(features)

In [10]:
features = np.concatenate([features_train, features_test], axis = 0)

In [12]:
features = preprocessing.scale(features)

In [14]:
features_train, features_test = features[:183143], features[183143:]

In [29]:
forest = RandomForestClassifier(100)

In [30]:
forest.fit(features_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [15]:
features = []
features.append(x_test[:, :11])
features = np.array(features)[0]
waves    = get_waves(x_test[:, 11:])
waves    = np.transpose(waves)
features = np.concatenate([features, waves], axis = 1)
features = preprocessing.scale(features)

In [31]:
y_pred  = forest.predict(features_test)

In [32]:
np.sum(y_pred == 0)/len(y_test)

0.5974697736046171

In [34]:
confusion_matrix(y_test, y_pred)

array([[26920,  4761,  2910],
       [11311,  6925,  5143],
       [ 8665,  5944,  5912]], dtype=int64)